<a href="https://colab.research.google.com/github/JaredLevi18/Stock-market-predictor./blob/main/model_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=d539d914fa10cca996d8a8b11414f5cae9a4f4dc91b09b67c8f0643e133cd9fe
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import pandas_ta as ta
import pandas as pd
import yfinance as yf

In [3]:
# donwload the data
company = input("Enter the desired stock: ")
def get_data(company):
  # downloading the data
  data = yf.download(tickers=company, start="2012-01-01", end="2023-09-30")

  # adding indicators with the help of pandas_ta
  data['RSI'] = ta.rsi(data.Close, length=15)

  data['EMAF'] = ta.ema(data.Close, length=20)
  data['EMAM'] = ta.ema(data.Close, length=100)
  data['EMAS'] = ta.ema(data.Close, length=150)

  data['Target'] = data['Adj Close']-data.Open
  data['Target'] = data['Target'].shift(-1)

  data['TargetClass'] = [1 if data.Target[i]>0 else 0 for i in range(len(data))]
  data['TargetNextClose'] = data['Adj Close'].shift(-1)

  # cleaning data
  data.dropna(inplace=True)
  data.reset_index(inplace=True)
  data.drop(['Volume', 'Close', 'Date'], axis=1, inplace=True)

  dataset = data.iloc[:, 0:11]

  return dataset

pd.set_option('display.max_columns', None)
dataset = get_data(company)
dataset[:-1]

Enter the desired stock: BTC-USD
[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Adj Close,RSI,EMAF,EMAM,EMAS,Target,TargetClass,TargetNextClose
0,221.968994,240.259003,221.261993,235.427002,50.019072,229.312807,288.640505,327.932773,21.793015,1,257.321014
1,235.528000,259.808014,235.528000,257.321014,58.174526,231.980256,288.020317,326.997518,-22.681992,0,234.824997
2,257.506989,265.610992,227.684006,234.824997,49.315763,232.251183,286.966944,325.776690,-0.981995,0,233.843002
3,234.824997,239.520996,229.022003,233.843002,48.967016,232.402785,285.914985,324.559025,10.188004,1,243.610001
4,233.421997,245.774994,232.313995,243.610001,52.543343,233.470139,285.077263,323.486853,-7.453995,0,236.326004
...,...,...,...,...,...,...,...,...,...,...,...
3144,26578.556641,26634.185547,26520.519531,26579.390625,49.610250,26517.455444,27417.818231,27265.873348,-322.546875,0,26256.826172
3145,26579.373047,26716.058594,26221.050781,26256.826172,45.484987,26492.633609,27394.828289,27252.508485,44.705078,1,26298.480469
3146,26253.775391,26421.507812,26011.468750,26298.480469,46.105048,26474.142833,27373.118431,27239.872352,-77.507812,0,26217.250000
3147,26294.757812,26389.884766,26090.712891,26217.250000,45.034788,26449.676849,27350.229947,27226.327685,143.218750,1,26352.716797


In [4]:
from pandas.core.interchange.dataframe_protocol import DataFrame
# normalizing data
dataset = dataset.to_numpy() # no longer a pandas.DataFrame, now is a numpy array.
class normalizer():
  def __init__(self) -> None:
    super().__init__()
    self.mean = None
    self.std = None

  def fit_transform(self, x):
    self.mean = np.mean(x, axis=(0), keepdims=True)
    self.std = np.std(x, axis=(0), keepdims=True)
    normalized_x = (x - self.mean) / self.std
    return normalized_x

  def denormalize_data(self, x):
    return (x * self.std) + self.mean

scaler = normalizer()
normalized_data = scaler.fit_transform(dataset)
normalized_data.shape

(3150, 11)

In [5]:
X = []

backcandels = 30
for j in range(8):
  X.append([])
  for i in range(backcandels, normalized_data.shape[0]):
    X[j].append(normalized_data[i-backcandels:i,j])

# move axis from 0 to position 2
X = np.moveaxis(X, [0], [2])

X, yi = np.array(X), np.array(normalized_data[backcandels:,-1])  # remove some of the data to match X shape
y = np.reshape(yi, (len(yi),1))
X.shape, y.shape

((3120, 30, 8), (3120, 1))

In [6]:
# generating val and training datasets
def split(X, y):
  spitlimit = int(len(X) * 0.9)
  # No longer a numpy.array, now a torch.tensor
  X_train, X_val = torch.from_numpy(X[:spitlimit]), torch.from_numpy(X[spitlimit:])
  y_train, y_val = torch.from_numpy(y[:spitlimit]), torch.from_numpy(y[spitlimit:])
  return X_train, X_val, y_train, y_val

X_train, X_val, y_train, y_val = split(X, y)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

(torch.Size([2808, 30, 8]),
 torch.Size([312, 30, 8]),
 torch.Size([2808, 1]),
 torch.Size([312, 1]))

In [7]:
X_train.shape[1]

30

In [22]:
#hyperparameters
input_size = X_train.shape[-1] * X_train.shape[1]
hidden_state = X_train.shape[1]
num_layers = 4
output_size = 1
dropout = 0.2

class Model(nn.Module):
  def __init__(self, input_size, hidden_state, num_layers, output_size, dropout) -> None:
    super().__init__()
    self.input_size = input_size
    self.hidden_state = hidden_state
    self.linear = nn.Linear(input_size, hidden_state)  # all the dimensions except the last one are the same, the last one is replaced for hidden_state
    self.batchnorm  = nn.BatchNorm1d(hidden_state)
    self.relu = nn.ReLU()
    self.lstm = nn.LSTM(hidden_state, hidden_size=500, num_layers=num_layers, batch_first=True)
    self.dropout = nn.Dropout(dropout)
    self.linear2 = nn.Linear(num_layers*hidden_state, output_size)
    self.init_weights()

  def init_weights(self):
    for name, param in self.lstm.named_parameters():
      if 'bias' == name:
        nn.init.constant_(param, 0.0)
      elif 'weight_ih' == name:
        nn.init.kaiming_normal_(param)
      elif 'weight_hh' in name:
        nn.init.orthogonal_(param)

  def forward(self, x):
    sequential = nn.Sequential(
        self.linear(),
        self.batchnorm(),
        self.relu()
    )
    result = sequential(x)
    lstm_out, (h_n, c_n) = self.dropout(self.lstm(result))
    x = lstm_out[:, -1]
    x = self.linear2(x)

    return x


model = Model(input_size, hidden_state, num_layers, output_size, dropout)
parameters = model.parameters()
print(f'Number of parameters: {sum(n.numel() for n in model.parameters())/ 1000000} million')
for p in parameters: p.requires_grad = True

Number of parameters: 21.321681 million


In [24]:
# training
max_steps = 10
batch_size = 32
lossi = []
model.double()

# training loop
for i in range(max_steps):
  # mini-batch construct
  ix = torch.randint(0, X_train.shape[0], (batch_size,))
  Xb, Yb = X_train[ix], y_train[ix]

  X_train = X_train.reshape(X_train.shape[0], -1)  # important to flatten the tensor  into a 2D for nn.Linear
  logits = model(X_train)
  loss = F.cross_entropy(logits, Yb)

  # backward pass
  for p in parameters: p.grad = None
  loss.backward()

  # update SGD:
  lr =  0.3 if i < (max_steps/2) else 0.005
  for p in parameters:
    p.data += -lr * p.grad
  optim.AdamW(params=parameters)

  # track of stats
  if i % 1 == 0:
    print(f'Step: {i:4f}/ {max_steps:7f}, {loss.item():4f}')
  lossi.append(loss.log10().item())

SyntaxError: ignored